In [1]:
import os

import numpy as np
import pandas as pd

import torch
import torchvision

import matplotlib.pyplot as plt
import lightning as L

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from data_module.data_module import FFTDataModule
from model.AE_model import AE1D_simple

c:\Users\1001002996\AppData\Local\miniconda3\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
TEST = True

random_seed = 42
fft_dir_path = "dataset/processed_data"

L.seed_everything(random_seed)

Seed set to 42


42

In [4]:
if TEST:
    data_module = FFTDataModule(dataset_path=fft_dir_path)
    data_module.setup()

    print(len(data_module.train_set))
    print(len(data_module.val_set))
    print(len(data_module.test_set))

    val_loader = data_module.val_dataloader()
    print(next(iter(val_loader)))

In [6]:
if TEST:
    from lightning.pytorch.utilities.model_summary import ModelSummary

    net = AE1D_simple(
        optimizer=optim.SGD,
        optimizer_param={
            "learning_rate": 0.01,
            "momentum": 0.5,
        })
    model_summary = ModelSummary(net, max_depth=6)

    print(model_summary)

   | Name         | Type            | Params | In sizes     | Out sizes   
--------------------------------------------------------------------------------
0  | encoder      | Sequential      | 18.0 K | [10, 6, 257] | [10, 64, 26]
1  | encoder.0    | Conv1d          | 1.6 K  | [10, 6, 257] | [10, 32, 84]
2  | encoder.1    | ReLU            | 0      | [10, 32, 84] | [10, 32, 84]
3  | encoder.2    | Dropout         | 0      | [10, 32, 84] | [10, 32, 84]
4  | encoder.3    | Conv1d          | 16.4 K | [10, 32, 84] | [10, 64, 26]
5  | encoder.4    | ReLU            | 0      | [10, 64, 26] | [10, 64, 26]
6  | encoder.5    | Dropout         | 0      | [10, 64, 26] | [10, 64, 26]
7  | enc_linear   | Sequential      | 2.0 M  | [10, 1664]   | [10, 256]   
8  | enc_linear.0 | Linear          | 1.7 M  | [10, 1664]   | [10, 1024]  
9  | enc_linear.1 | ReLU            | 0      | [10, 1024]   | [10, 1024]  
10 | enc_linear.2 | Linear          | 262 K  | [10, 1024]   | [10, 256]   
11 | dec_linear   |

In [15]:
n_epochs = 10000
patience = 100

In [16]:
print(" ----------------------start training---------------------------")
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger, CSVLogger

tensorboard_logger = TensorBoardLogger(
    save_dir="lightning_logs",
    name="AE1D_lin_gyr",
)

csv_logger = CSVLogger(
    save_dir="lightning_logs",
    name="AE1D_lin_gyr",
)

checkpoint_callback = ModelCheckpoint(
    dirpath=None,
    save_top_k=1,
    monitor="val_mse",
    mode="min",
    filename="sample_{epoch:02d}-{step:02d}-{val_mse:02f}"
)

trainer = L.Trainer(
    logger=[tensorboard_logger, csv_logger],
    callbacks=[EarlyStopping(monitor="val_mse", patience=patience), checkpoint_callback],
    max_epochs=n_epochs,
    check_val_every_n_epoch=10
    # fast_dev_run=True,
    )

net = AE1D_simple(
    optimizer=optim.SGD,
    optimizer_param={
        "learning_rate": 0.01,
        "momentum": 0.5,
    })
data_module = FFTDataModule(dataset_path=fft_dir_path, batch_size=8192)

trainer.fit(model=net, datamodule=data_module)
trainer.test(model=net, datamodule=data_module)

 ----------------------start training---------------------------


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: lightning_logs\AE1D


c:\Users\1001002996\AppData\Local\miniconda3\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:639: Checkpoint directory C:\Users\1001002996\Documents\AE\lightning_logs exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params | In sizes     | Out sizes   
------------------------------------------------------------------------
0 | encoder    | Sequential | 18.0 K | [10, 6, 257] | [10, 64, 26]
1 | enc_linear | Sequential | 2.0 M  | [10, 1664]   | [10, 256]   
2 | dec_linear | Sequential | 2.0 M  | [10, 256]    | [10, 1664]  
3 | decoder    | Sequential | 18.0 K | [10, 64, 26] | [10, 6, 257]
------------------------------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
15.888    Total estimated model params size (MB)


c:\Users\1001002996\AppData\Local\miniconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\1001002996\AppData\Local\miniconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 2:   1%|          | 3/306 [00:00<00:04, 75.43it/s, v_num=0, train_mse=0.0645]  

c:\Users\1001002996\AppData\Local\miniconda3\Lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\1001002996\AppData\Local\miniconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 39/39 [00:00<00:00, 194.39it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_mse            0.06475704908370972
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_mse': 0.06475704908370972}]